In [1]:
import serial
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from time import sleep

In [2]:
def getFloat(fromserial):
    for ele in fromserial:
        try:
            number=float(ele)
        except:
            pass
    return number

def getInt(fromserial):
    for ele in fromserial:
        try:
            number=int(ele)
        except:
            pass
    return number

def getChunks(hexes, size,vertgain,vertoff):
    chunks = [hexes[i:i+size] for i in range(0,len(hexes),size)]
    #print(chunks)
    vals =[]
    for ele in chunks:
        hhex = bytes.fromhex(ele)
        hint = int.from_bytes(hhex,byteorder='big',signed=True)
        vals.append(vertgain*hint+vertoff)
    return vals

def getWaveformParams(serialobject):
    flushAll(serialobject)
    serialobject.write(b'TA:INSPECT? \"WAVE_ARRAY_COUNT\"\r')
    counts= getInt(serialobject.read_until(expected=b'\n\r').decode('utf-8').split())
    print('Counts:',counts)

    flushAll(serialobject)
    serialobject.write(b'TA:INSPECT? \"VERTICAL_GAIN\"\r')
    vertgain = getFloat(serialobject.read_until(expected=b'\n\r').decode('utf-8').split())
    print('Vertical gain:',vertgain)
    
    flushAll(serialobject)
    serialobject.write(b'TA:INSPECT? \"VERTICAL_OFFSET\"\r')
    vertoff = getFloat(serialobject.read_until(expected=b'\n\r').decode('utf-8').split())
    print('Vertical offset',vertoff)

    flushAll(serialobject)
    serialobject.write(b'TA:INSPECT? \"HORIZ_INTERVAL\"\r')
    horint = getFloat(serialobject.read_until(expected=b'\n\r').decode('utf-8').split())
    print('Horizontal interval:',horint)

    flushAll(serialobject)
    serialobject.write(b'TA:INSPECT? \"HORIZ_OFFSET\"\r')
    horoff = getFloat(serialobject.read_until(expected=b'\n\r').decode('utf-8').split())
    print('Horizontal offset:',horoff)
    return counts,vertgain,vertoff,horint,horoff
    
def flushAll(serialobject):
    serialobject.flushInput()
    serialobject.flushOutput()

In [10]:
def waitReady(sleept,timeout=5,verbose=0):
    t1 = dt.datetime.now()
    d=0
    c=0
    lecroy.write(b'*CLS\r')
    out = []
    while (c!=257) and (d< timeout/sleept):
        lecroy.write(b'INR?\r')
        output=lecroy.read_until(expected=b'\n\r').decode('utf-8').split()
        if verbose !=0:
            out.append(output)
        try:
            c=getInt(output)
        except:
            pass
        if d%100==0:
            ttemp = dt.datetime.now()
            print(d,(ttemp-t1).total_seconds())
    
        d +=1
        sleep(sleept)
    
    print(c,d)
    t2 = dt.datetime.now()
    print((t2-t1).total_seconds())
    if verbose !=0:
        print(out)

In [5]:
lecroy = serial.Serial('COM2',115200,timeout=10)
scanmate = serial.Serial('COM1',9600,stopbits = serial.STOPBITS_TWO,timeout=2)


In [6]:
lecroy.write(b'DISP OFF\r')
lecroy.write(b'COMM_HEADER OFF\r')
lecroy.write(b'COMM_FORMAT OFF,WORD,HEX\r')
flushAll(lecroy)
lecroy.write(b'ALST?\r')
lecroy.read_until(expected=b'\n\r')

b'\r\nCOMM_FORMAT OFF,WORD,H\r\nSTB,000000,ESR,000000,INR,008193,DDR,000000,CMR,000000,EXR,000000,URR,000000\n\r'

In [7]:
size = 4 ### word= 2 bytes=4 hex, 1 byte=2 hex 
counts,vertgain,vertoff,horint,horoff=getWaveformParams(lecroy)
tdivs=np.arange(counts)*horint+horoff

Counts: 5002
Vertical gain: 6.1035e-06
Vertical offset -0.044
Horizontal interval: 2e-05
Horizontal offset: -1e-05


In [41]:
lecroy.write(b'TA:FRST\r')
waitReady(.01,verbose=0)

0 0.001004
257 58
0.629362


In [43]:
for i in range(5):
    print('Run:',i+2)
    t1 = dt.datetime.now()
    lecroy.write(b'STO TA,M1\r')
    lecroy.write(b'TA:FRST\r')
    flushAll(lecroy)
    lecroy.write(b'M1:WF? DAT1\r')
    #data=lecroy.read_until(expected=b'\n\r').decode('utf-8').split()
    t2 = dt.datetime.now()
    print((t2-t1).total_seconds())
    #print(data[-1])
    values = getChunks(data[-1],size,vertgain,vertoff)
    t3 = dt.datetime.now()
    print((t3-t1).total_seconds())
    #sleep(.8)
    waitReady(.01)
    t4 = dt.datetime.now()
    print((t4-t1).total_seconds())

Run: 2
0.002005
0.008017
0 0.002004
100 1.12649
200 2.244829
300 3.368812
257 349
3.919757
3.927774
Run: 3
0.001718
0.015215
0 0.002344
100 1.150688
200 2.267772
300 3.395315
257 352
3.975514
3.991244
Run: 4
0.002013
0.016529
0 0.030313
100 1.414152
200 2.808952
257 278
3.892874
3.909403
Run: 5
0.002257
0.020731
0 0.020384
100 1.437644
200 2.838483
257 282
3.985535
4.006266
Run: 6
0.002189
0.01927
0 0.018545
100 1.403942
200 2.810563
257 284
3.970703
3.990481


In [ ]:
lecroy.close()
scanmate.close()

In [25]:
lecroy.write(b'DISP ON\r')
flushAll(lecroy)

In [50]:
t1 = dt.datetime.now()
flushAll(lecroy)
lecroy.write(b'TA:WF? DAT1\r')
t3 = dt.datetime.now()
data=lecroy.read_until(expected=b'\n\r').decode('utf-8').split()
t2 = dt.datetime.now()
print((t2-t1).total_seconds())
print((t2-t3).total_seconds())

1.733841
1.733333


In [51]:
data

['TA:WF?',
 'D',
 '...']

In [52]:
lecroy.read(10)

b''